In [4]:
import os

from embed_time.model_VAE_resnet18 import VAEResNet18
import torch
from torch.utils.data import DataLoader
from torch.nn import functional as F
from torch.nn import utils as U
from torch import optim
from torchvision.transforms import v2
import matplotlib.pyplot as plt
import subprocess
import pandas as pd
import numpy as np
from torch.utils.tensorboard import SummaryWriter
from datetime import datetime
import yaml
from datasets.neuromast import NeuromastDatasetTrain, NeuromastDatasetTest, NeuromastDatasetTrain_T10
from torchview import draw_graph
#%%


beta = 1e-4
lr = 1e-3
z_dim = 22
model_name = "neuromast_resnet18_vae_conv2D"
run_name= "z_dim-"+str(z_dim)+"_lr-"+str(lr)+"_beta-"+str(beta)
metadata = pd.read_csv("/mnt/efs/dlmbl/G-et/data/neuromast/Dataset/metadata_neuromast_balanced_train.csv")

if torch.cuda.is_available():
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

#launch tensorboard



#%% Generate Dataset
dataset = NeuromastDatasetTrain_T10()

#dataloader
train_loader = DataLoader(dataset, batch_size=1, shuffle=True, num_workers=8)

# Initiate VAE-ResNet18 model
vae = VAEResNet18(nc = 1, z_dim = z_dim ).to(device)

#%% Define Optimizar
optimizer = torch.optim.AdamW(vae.parameters(), lr=lr)

#%% Define loss function
def loss_function(recon_x, x, mu, logvar):
    MSE = F.mse_loss(recon_x, x, reduction='mean')
    KLD = -0.5 * torch.sum(1 + logvar - mu.pow(2) - logvar.exp())
    return MSE, KLD   
import torch
from torchviz import make_dot
import torch.nn.functional as F

# Assuming your VAEResNet18_3D model is defined as `VAEResNet18_3D`
# Initialize the model
model = VAEResNet18(nc=1, z_dim=10)

#import torch
from torchviz import make_dot
import torch.nn.functional as F

# Assuming your VAEResNet18_3D model is defined as `VAEResNet18_3D`
# Initialize the model
model = VAEResNet18(nc=1, z_dim=22)

# Create a sample input tensor with shape (1, 1, 64, 256, 256)
input_tensor,label = dataset[11] # Adjust nc=1 for single channel input
input_tensor = torch.from_numpy(input_tensor)
input_tensor = input_tensor.unsqueeze(0)

# Pass the sample input through the model
output, mu, logvar = model(input_tensor)

graph = draw_graph(model, input_tensor)
graph.visual_graph

RuntimeError: Calculated padded input size per channel: (2 x 2). Kernel size: (3 x 3). Kernel size can't be greater than actual input size

In [6]:
from iohub.ngff import open_ome_zarr
from natsort import natsorted
from glob import glob
from pathlib import Path 
import torch
from torch.utils.data import Dataset
from scipy.ndimage import measurements
from scipy.ndimage import center_of_mass
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [7]:
file_path = "/mnt/efs/dlmbl/G-et/data/neuromast/Dataset/"
zarr_file = 'structured_celltype_classifier_data.zarr/*/*/*'
position_paths = natsorted(glob(file_path + zarr_file))
position_paths = position_paths[:500]
cell_count = 40 # number of cells to sample from each timepoint
print(position_paths[0])
print(open_ome_zarr(position_paths[0]).data. shape)


/mnt/efs/dlmbl/G-et/data/neuromast/Dataset/structured_celltype_classifier_data.zarr/0/0/0
(1, 4, 73, 1024, 1024)


In [8]:
shape = open_ome_zarr(position_paths[0]).data.shape
print(shape[3])

1024


In [9]:
# Find the maximum range across all dimensions
max_x_range = 256
max_y_range = 256
max_z_range = 48 # not used for cropping

crop_size = [max_z_range, max_y_range, max_x_range]
print("crop_size: ", crop_size)

row = metadata.iloc[0]
# Get centroid coordinates
centroid_z = int(row['Centroid_Z'])
centroid_y = int(row['Centroid_Y'])
centroid_x = int(row['Centroid_X'])
print("centroid_z: ", centroid_z, "centroid_y: ", centroid_y, "centroid_x: ", centroid_x)

#get the label number
label = int(row['Label'])
print("label: ", label)
timepoint = int(row['T_value'])
print("timepoint: ", timepoint)

# Compute the cropping box boundaries
z_min = int(row['Z_min'])
z_max = int(row['Z_max'])
y_min = int(max((int(centroid_y - crop_size[1] // 2)),0))
y_max = int(min((int(centroid_y + crop_size[1] // 2)), shape[3]-1))
x_min = int(max((int(centroid_x - crop_size[2] // 2)), 0))
x_max = int(min((int(centroid_x + crop_size[2] // 2)), shape[4]-1))

mid_z = (z_min + z_max) // 2
print("mid_z: ", mid_z)
print("y_min: ", y_min, "y_max: ", y_max)
print("x_min: ", x_min, "x_max: ", x_max)

# Load the corresponding image from the dataset (assuming 5D dataset [T, C, Z, Y, X])
dataset = open_ome_zarr(position_paths[timepoint], mode="r")
image = dataset.data[0,0:1,mid_z,y_min:y_max, x_min:x_max]
print(image.shape)
 segmented_data = dataset.data[0,2:3,mid_z,y_min:y_max, x_min:x_max] #segmention masks
        # celltypes = dataset.data[0,3:,:,:,:]
        # Get a binary mask of the current segment
        segment_mask = segmented_data == label
        

        # Find the unique label numbers in the celltypes image for this segment
        cell_type = int(row['Cell_Type'])
        cropped_image=np.where(segment_mask, image, 0)
        
        
        return cropped_image, cell_type

crop_size:  [48, 256, 256]
centroid_z:  27 centroid_y:  427 centroid_x:  420
label:  19
timepoint:  0
mid_z:  28
y_min:  299 y_max:  555
x_min:  292 x_max:  548
(1, 256, 256)


In [11]:
# Function to find an available port
def find_free_port():
    import socket

    with socket.socket(socket.AF_INET, socket.SOCK_STREAM) as s:
        s.bind(("", 0))
        return s.getsockname()[1]


# Launch TensorBoard on the browser
def launch_tensorboard(log_dir):
    port = find_free_port()
    tensorboard_cmd = f"tensorboard --logdir={log_dir} --port={port}"
    process = subprocess.Popen(tensorboard_cmd, shell=True)
    print(
        f"TensorBoard started at http://localhost:{port}. \n"
        "If you are using VSCode remote session, forward the port using the PORTS tab next to TERMINAL."
    )
    return process

# Launch tensorboard and click on the link to view the logs.

tensorboard_process = launch_tensorboard("embed_time_static_runs")

TensorBoard started at http://localhost:39227. 
If you are using VSCode remote session, forward the port using the PORTS tab next to TERMINAL.


TensorFlow installation not found - running with reduced feature set.

NOTE: Using experimental fast data loading logic. To disable, pass
    "--load_fast=false" and report issues on GitHub. More details:
    https://github.com/tensorflow/tensorboard/issues/4784

Serving TensorBoard on localhost; to expose to the network, use a proxy or pass --bind_all
TensorBoard 2.17.1 at http://localhost:39227/ (Press CTRL+C to quit)


In [13]:
dataset_train = NeuromastDatasetTrain_T10()

dataloader_train = DataLoader(dataset_train, batch_size=2, shuffle=True, num_workers=8)
len(dataloader_train.dataset)
len(dataloader_train)

153